# Processing the Joined Table Locally

Alternatively to processing the joined table using big data tools (eg., Spark, Athena/Presto) it's possible to inspect or process the table locally. See the [general instructions](../README.md)) how the joined table was created.

First, you need to copy the table to your local file system:
```
aws s3 sync s3://bucket/path/cc/ data-sourcing-sheet/cc/
```
Please change the local path (`data-sourcing-sheet/cc/`) if needed.

We now load the table from the local path, filtering on the fly only successfully fetched URLs (`subset=warc`):

In [1]:
import pyarrow as pa
import pyarrow.parquet as pq

dataset = pq.ParquetDataset('data-sourcing-sheet/cc/', filters=[('subset', '=', 'warc')])

dataset.schema

required group field_id=-1 hive_schema {
  optional int32 field_id=-1 id;
  optional binary field_id=-1 title (String);
  optional binary field_id=-1 link (String);
  optional binary field_id=-1 language (String);
  optional binary field_id=-1 url_surtkey_prefix (String);
  optional binary field_id=-1 url_surtkey (String);
  optional binary field_id=-1 url_host_tld (String);
  optional binary field_id=-1 url_host_registered_domain (String);
  optional binary field_id=-1 url_host_name (String);
  optional binary field_id=-1 url (String);
  optional int32 field_id=-1 fetch_status;
  optional int96 field_id=-1 fetch_time;
  optional binary field_id=-1 warc_filename (String);
  optional int32 field_id=-1 warc_record_offset;
  optional int32 field_id=-1 warc_record_length;
  optional binary field_id=-1 fetch_redirect (String);
  optional binary field_id=-1 content_mime_detected (String);
  optional binary field_id=-1 content_languages (String);
}

In [2]:
# read only two columns (the two partitioning columns "crawl" and "subset" are included as well)
table = dataset.read(columns=['id', 'url'])

In [3]:
table.shape

(35410516, 4)

In [4]:
table.column_names

['id', 'url', 'crawl', 'subset']

In [5]:
# iterate over values of a single column
for url in table['url']:
    print(url)
    break

https://www.latribunadealbacete.es/GaleriaDetalle.aspx?g=ZD56B811C-CEFD-3C9C-F5664489815E81EE&f=D57979D4-BB46-2EF1-CA526BDAAFC8364F


In [6]:
# convert to pandas DataFrame
df = table.to_pandas()

df.sample(10)

,id,url,crawl,subset
10828179,90,https://peru.com/actualidad/economia-y-finanza...,CC-MAIN-2020-34,warc
2135420,358,https://www.laverdad.es/loteriadenavidad/local...,CC-MAIN-2020-10,warc
8404237,22,https://www.clubinfluencers.com/los-esports-se...,CC-MAIN-2020-29,warc
28463197,396,https://www.eldiario.es/opinion/zona-critica/a...,CC-MAIN-2021-31,warc
31588496,281,https://www.cmdsport.com/nieve/competiciones-n...,CC-MAIN-2021-43,warc
32542315,276,http://radio.uchile.cl/2017/09/03/candidatos-a...,CC-MAIN-2021-43,warc
5759847,357,http://ntrzacatecas.com/temas/sezami/,CC-MAIN-2020-24,warc
25256106,503,https://www.zaobao.com.sg/keywords/mai-dang-lao,CC-MAIN-2021-21,warc
14956664,262,https://www.rtve.es/alacarta/videos/telediario...,CC-MAIN-2020-45,warc
35255396,325,https://www.laprensa.hn/honduras/omar-rivera-c...,CC-MAIN-2021-49,warc
